#### Imports

In [ ]:
!pip install python-dotenv

In [ ]:
import os
import requests

from dotenv import load_dotenv
from urllib.parse import urljoin
from urllib.parse import urlparse
from urllib.request import urlretrieve

#### Data directory

In [ ]:
data_directory_path = "../data"

#### Environmental variables loading

In [ ]:
env_filepath = "../../env/gov/dados.env"

In [ ]:
load_dotenv(env_filepath)

True

#### Request headers and base URL

In [ ]:
request_headers = {
    'accept': 'application/json',
    'chave-api-dados-abertos': os.environ["CHAVE_API_DADOS_GOV_BR"],
}

request_base_url = urlparse('https://dados.gov.br/dados/api/publico/')

In [ ]:
ufrn_id = 'c0dd9ea9-d067-4d8d-97d9-8016f29873aa'

In [ ]:
requested = requests.get(
    urljoin(request_base_url.geturl(),'organizacao/' + ufrn_id), 
    headers=request_headers, 
)

ufrn_information = requested.json()

In [ ]:
ufrn_information

{'id': 'c0dd9ea9-d067-4d8d-97d9-8016f29873aa',
 'name': None,
 'displayName': 'Universidade Federal do Rio Grande do Norte - UFRN',
 'urlFoto': 'https://dados.gov.br/api/publico/s3/2c29c868-2afb-4da9-8521-d7079ba9063a.png',
 'quantidadeSeguidores': 4,
 'quantidadeConjuntoDados': 88,
 'descricao': 'A Universidade Federal do Rio Grande do Norte origina-se da Universidade do Rio Grande do Norte, criada em 25 de junho de 1958, através de lei estadual, e federalizada em 18 de dezembro de 1960. A Universidade do Rio Grande do Norte, instalada em sessão solene realizada no Teatro Alberto Maranhão, em 21 de março de 1959, foi formada a partir de faculdades e escolas de nível superior já existentes em Natal, como a Faculdade de Farmácia e Odontologia; a Faculdade de Direito; a Faculdade de Medicina; a Escola de Engenharia, entre outras.',
 'nome': 'Universidade Federal do Rio Grande do Norte - UFRN',
 'ativo': 'active',
 'atributos': {'siorg': '417'}}

In [ ]:
conjunto_dados = []
pagina = 0
while True:
    pagina += 1
    print(pagina, end=', ')
    request_params = {
        'idOrganizacao' : ufrn_id,
        'pagina' : pagina,
    }
    
    request_url = "https://dados.gov.br/dados/api/publico/conjuntos-dados"
    requested = requests.get(request_url, headers=request_headers, params=request_params)

    if requested.json():
        conjunto_dados.extend(requested.json())
    else:
        break
ufrn_information['conjunto_dados'] = conjunto_dados

1, 2, 3, 4, 5, 6, 7, 

In [ ]:
ufrn_raw_directory = '../data/raw/universidade_federal_do_rio_grande_do_norte_-_ufrn/'

In [ ]:
for d, dado in enumerate(ufrn_information['conjunto_dados']):
    print('\t'+str(d), end=' (')
    request_url = "https://dados.gov.br/dados/api/publico/conjuntos-dados/" + dado['id']
    response = requests.get(request_url, headers=request_headers, params=request_params)
    response_json = response.json()
    if response_json:
        print(dado['title'], end=')\n')
        dado.update(response_json)

	0 (I Hackathon UFRN)
	1 (Relação Empenhos COVID inscritos em RPNP)
	2 (Acervo aquisição)
	3 (Servidores Aposentados)
	4 (Trabalhos de Conclusão de Curso)
	5 (Requisição de Serviço para Licitação)
	6 (Requisição de Parecer Técnico/Avaliação)
	7 (Requisição de Material para Importação)
	8 (Requisição de Serviço para Pessoa Jurídica)
	9 (Requisição de Materiais Informacionais do Contrato)
	10 (Requisição de Reserva de Veículo)
	11 (Requisição de Atividade de Campo)
	12 (Requisição de Meio Ambiente)
	13 (Requisição de Devolução de Material)
	14 (Solicitação de Material de Registro de Preço)
	15 (Requisições de Materiais)
	16 (Componentes Curriculares)
	17 (Requisição de Materiais Informacionais)
	18 (Orientações de Docentes)
	19 (Estruturas Curriculares)
	20 (Planos Individuais Docentes)
	21 (Funções Gratificadas)
	22 (AVALIAÇÕES DE DESEMPENHO DE DOCENTES)
	23 (Requisição de Material para Participar de um Novo Registro de Preço)
	24 (Dados Complementares de Discentes)
	25 (Unidades Acad

In [ ]:
should_break = False
for dataset in ufrn_information['conjunto_dados']:
    for recurso in dataset['recursos']:
        if not recurso['formato']: continue
        resource_filename = recurso['link'].split('/')[-1]
        resource_filepath = os.path.join(ufrn_raw_directory,resource_filename)
        if os.path.exists(resource_filepath): continue
        print(resource_filepath)
        urlretrieve(recurso['link'],resource_filepath)